<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset el libro de la republica de platon

In [2]:

with open('./dataset/the_republic.txt') as f:
    content = f.readlines()

content = [x.strip() for x in content] 
df = pd.DataFrame(content)
df.head()

,0
0,ï»¿The Project Gutenberg EBook of The Republic...
1,
2,This eBook is for the use of anyone anywhere i...
3,other parts of the world at no cost and with a...
4,"whatsoever. You may copy it, give it away or ..."


In [3]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 29809


### 1 - Preprocesamiento

In [4]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [5]:
# Demos un vistazo
sentence_tokens[:2]

[['ï»¿the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'the',
  'republic',
  'of',
  'plato',
  'by',
  'plato'],
 []]

### 2 - Crear los vectores (word2vec)

In [6]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [7]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=600,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=3,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [8]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [9]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 29809


In [10]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

### 3 - Entrenar el modelo generador

In [11]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 661435.375
Loss after epoch 1: 520184.625
Loss after epoch 2: 467002.0
Loss after epoch 3: 461811.5
Loss after epoch 4: 409495.25
Loss after epoch 5: 407040.75
Loss after epoch 6: 402421.0
Loss after epoch 7: 397382.25
Loss after epoch 8: 397290.5
Loss after epoch 9: 370586.25
Loss after epoch 10: 366384.5
Loss after epoch 11: 359934.0
Loss after epoch 12: 358997.5
Loss after epoch 13: 356032.5
Loss after epoch 14: 353538.5
Loss after epoch 15: 352286.5
Loss after epoch 16: 348761.0
Loss after epoch 17: 351177.0
Loss after epoch 18: 348377.5
Loss after epoch 19: 348323.5


(3329558, 5170680)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["republic"], topn=10)

[('analysis', 0.6907597780227661),
 ('introduction', 0.6811822652816772),
 ('books', 0.6379269957542419),
 ('recapitulation', 0.6356292963027954),
 ('book', 0.6238475441932678),
 ('cicero', 0.6202841997146606),
 ('illustrations', 0.6078434586524963),
 ('odyssey', 0.6017596125602722),
 ('consistency', 0.5938580632209778),
 ('fifth', 0.5918599963188171)]

In [13]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('how', 0.0008103308500722051),
 ('me', -0.02552296221256256),
 ('two', -0.03827720880508423),
 ('we', -0.05737166479229927),
 ('first', -0.057613566517829895),
 ('different', -0.06142646074295044),
 ('was', -0.06203175336122513),
 ('further', -0.06314301490783691),
 ('going', -0.06742909550666809),
 ('what', -0.06754294037818909)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["achilles"], topn=10)

[('phoenix', 0.8176901340484619),
 ('apollo', 0.8032463788986206),
 ('herã¨', 0.7860184907913208),
 ('391', 0.7854374647140503),
 ('zeus', 0.7790477871894836),
 ('chryses', 0.7762696146965027),
 ('peirithous', 0.7708088755607605),
 ('patroclus', 0.7677872776985168),
 ('388', 0.7651506066322327),
 ('392', 0.7642678618431091)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["socrates"], topn=5)

[('interposing', 0.6217087507247925),
 ('thrasymachus', 0.6173322200775146),
 ('cleitophon', 0.6075360774993896),
 ('cephalus', 0.5973382592201233),
 ('adeimantus', 0.5817573070526123)]

In [21]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["cicero"], topn=5)

[('eristic', 0.8113694787025452),
 ('detained', 0.8089227080345154),
 ('translated', 0.8029212951660156),
 ('rebuked', 0.7995917201042175),
 ('characterized', 0.797544538974762)]

In [15]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

### 5 - Visualizar agrupación de vectores

In [18]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key )  

    tsne = TSNE(n_components=num_dimensions, random_state=0, n_jobs= -1)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

# Cambios realizados

- Se utilizo el libro de la republica de platon como dataset, extraido de gutenberg project.
- Se incremento el numero de la dimension de embeddings, al hacer este ajuste, el loss disminuye considerablemente. 
- Se probaron terminos de interes como, republic, achilles, etc, y se encontraron las palabras con mayor similitud.

# Conclusiones

- La dimension de los embeddings, cambian considerablemente el loss de entrenamiento.
- Al buscar las palabras similares a "achilles" se encontro que los embeddings mapearon a otros personajes de la historia.
